In [1]:
library(tidyverse)
library(RPostgreSQL)
library(tableone)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
data_dir <- file.path("..", "data")
list.files(sql_dir)

[1] "basics.sql"      "cohort.sql"      "lab_tests.sql"   "population.sql" 
[5] "vital_signs.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- "
select * from basics;
"
basics <- dbGetQuery(con, sql) %>%
    mutate(weekday = case_when(
        weekday <= 0 ~ "sunday",
        weekday <= 1 ~ "monday",
        weekday <= 2 ~ "tuesday",
        weekday <= 3 ~ "wednesday",
        weekday <= 4 ~ "thursday",
        weekday <= 5 ~ "friday",
        weekday <= 6 ~ "saturday"
    ))
head(basics)

icustay_id,hadm_id,subject_id,age,echo,gender,weight,saps,sofa,elix_score,vent,vaso,weekday
200003,163557,27513,48.29627,1,M,77.0,24,6,0,1,0,friday
200003,163557,27513,48.29627,1,M,77.0,24,6,0,1,0,friday
200003,163557,27513,48.29627,1,M,77.0,24,6,0,1,1,friday
200014,127229,9514,84.73004,1,M,62.0,23,3,3,1,0,monday
200030,165426,14987,54.19308,1,M,113.6,17,7,19,0,1,friday
200033,198650,56369,67.14657,0,M,74.0,15,1,9,0,0,tuesday


In [6]:
tab <- CreateTableOne(vars = c("age", "gender", "weight", "elix_score",
                               "saps", "sofa", "vent", "vaso", "weekday"),
                      strata = "echo",
                      factorVars = c("gender", "vent", "vaso", "weekday"),
                      data = basics) %>%
    print

                        Stratified by echo
                         0             1             p      test
  n                       3905          6004                    
  age (mean (sd))        82.67 (64.70) 78.77 (59.13)  0.002     
  gender = M (%)          1913 (49.0)   3223 (53.7)  <0.001     
  weight (mean (sd))     80.00 (24.56) 83.18 (27.50) <0.001     
  elix_score (mean (sd))  8.70 (7.54)  10.33 (7.61)  <0.001     
  saps (mean (sd))       20.03 (5.73)  21.73 (5.45)  <0.001     
  sofa (mean (sd))        5.69 (3.79)   7.02 (3.98)  <0.001     
  vent = 1 (%)            2070 (53.0)   4028 (67.1)  <0.001     
  vaso = 1 (%)            1014 (26.0)   1496 (24.9)   0.250     
  weekday (%)                                        <0.001     
     friday                708 (18.1)    824 (13.7)             
     monday                494 (12.7)    866 (14.4)             
     saturday              547 (14.0)    833 (13.9)             
     sunday                504 (12.9)    850 (1

In [7]:
write.csv(tab, file.path(data_dir, "basics_tbone.csv"))

In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE